In [8]:
import numpy as np
import xarray as xr
import scipy.fftpack as fft
import dask.array as dsar
import os.path as op
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
def synthetic_field(N=512, dL=1., amp=1e1, s=-3.):
    """
    Generate a synthetic series of size N by N 
    with a spectral slope of s.
    """

    k = fft.fftshift(fft.fftfreq(N, dL))
    l = fft.fftshift(fft.fftfreq(N, dL))
    kk, ll = np.meshgrid(k, l)
    K = np.sqrt(kk**2+ll**2)

    ########
    # amplitude
    ########
    r_kl = np.ma.masked_invalid(np.sqrt(amp*.5*(np.pi)**(-1)*K**(s-1.))).filled(0.)
    #r = np.ma.masked_invalid(np.abs(k)**(-slope/2.)).filled(0.)
    ########
    # phase
    ########
    phi = np.zeros((N, N))
    
    N_2 = int(N/2)
    phi_upper_right = 2.*np.pi*np.random.random((N_2-1, 
                                                 N_2-1)) - np.pi
    phi[N_2+1:,N_2+1:] = phi_upper_right.copy()
    phi[1:N_2, 1:N_2] = -phi_upper_right[::-1, ::-1].copy()


    phi_upper_left = 2.*np.pi*np.random.random((N_2-1, 
                                                N_2-1)) - np.pi
    phi[N_2+1:,1:N_2] = phi_upper_left.copy()
    phi[1:N_2, N_2+1:] = -phi_upper_left[::-1, ::-1].copy()


    phi_upper_middle = 2.*np.pi*np.random.random(N_2) - np.pi
    phi[N_2:, N_2] = phi_upper_middle.copy()
    phi[1:N_2, N_2] = -phi_upper_middle[1:][::-1].copy()


    phi_right_middle = 2.*np.pi*np.random.random(N_2-1) - np.pi
    phi[N_2, N_2+1:] = phi_right_middle.copy()
    phi[N_2, 1:N_2] = -phi_right_middle[::-1].copy()


    phi_edge_upperleft = 2.*np.pi*np.random.random(N_2) - np.pi
    phi[N_2:, 0] = phi_edge_upperleft.copy()
    phi[1:N_2, 0] = -phi_edge_upperleft[1:][::-1].copy()


    phi_bot_right = 2.*np.pi*np.random.random(N_2) - np.pi
    phi[0, N_2:] = phi_bot_right.copy()
    phi[0, 1:N_2] = -phi_bot_right[1:][::-1].copy()


    phi_corner_leftbot = 2.*np.pi*np.random.random() - np.pi


#     print(phi[N/2-1,N-1], phi[N/2+1,1])
#     print(phi[N/2+1,N/2+1], phi[N/2-1,N/2-1])


#     phi[:N/2, :] = -np.rot90(np.rot90(phi[N/2:, :]))
#     phi[:N/2, :] = -phi[N/2:, :][::-1,::-1]
#     i, j = 25, 40
#     print(phi[N/2+j,N/2+i], -phi[N/2-j,N/2-i])

    for i in range(1, N_2):
        for j in range(1, N_2):
            assert (phi[N_2+j, N_2+i] == -phi[N_2-j, N_2-i])
        
    for i in range(1, N_2):
        for j in range(1, N_2):
            assert (phi[N_2+j, N_2-i] == -phi[N_2-j, N_2+i])
        
    for i in range(1, N_2):
        assert (phi[N_2, N-i] == -phi[N_2, i])
        assert (phi[N-i, N_2] == -phi[i, N_2])
        assert (phi[N-i, 0] == -phi[i, 0])
        assert (phi[0, i] == -phi[0, N-i])
    #########
    # complex fourier amplitudes
    #########
    #a = r + 1j*th
    F_theta = r_kl * np.exp(1j * phi)
    
    # check that symmetry of FT is satisfied
    #np.testing.assert_almost_equal(a[1:N/2], a[-1:-N/2:-1].conj())

    theta = fft.ifft2(fft.ifftshift(F_theta))
    return np.real(theta) 

In [97]:
theta = synthetic_field(N=512, s=-3.)
print(theta.shape)
theta = xr.DataArray(theta, dims=['x', 'y'],
            coords={'x':range(512), 'y':range(512)})
# theta = dsar.from_array(theta, chunks=(32, 32))
theta

(512, 512)


/Users/uchidatakaya/anaconda/envs/xrft/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in power
  from ipykernel import kernelapp as app


<xarray.DataArray (x: 512, y: 512)>
array([[ 1.33968 ,  1.244857,  1.141161, ...,  1.700922,  1.556506,  1.456991],
       [ 1.353033,  1.239265,  1.14533 , ...,  1.630566,  1.474879,  1.413957],
       [ 1.395392,  1.294711,  1.158358, ...,  1.604388,  1.473878,  1.416435],
       ..., 
       [ 1.407155,  1.276178,  1.131   , ...,  1.761787,  1.645072,  1.53301 ],
       [ 1.373473,  1.259248,  1.115498, ...,  1.756696,  1.65175 ,  1.508392],
       [ 1.378717,  1.269396,  1.149053, ...,  1.730754,  1.603561,  1.485835]])
Coordinates:
  * x        (x) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * y        (y) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...

In [98]:
def dft(da, dim=None, shift=True, remove_mean=True):
    """
    Perform discrete Fourier transform of xarray data-array `da` along the
    specified dimensions.

    Parameters
    ----------
    da : `xarray.DataArray`
        The data to be transformed
    dim : list (optional)
        The dimensions along which to take the transformation. If `None`, all
        dimensions will be transformed.
    shift : bool (optional)
        Whether to shift the fft output.
    remove_mean : bool (optional)
        If `True`, the mean across the transform dimensions will be subtracted
        before calculating the Fourier transform.

    Returns
    -------
    daft : `xarray.DataArray`
        The output of the Fourier transformation, with appropriate dimensions.
    """

    if dim is None:
        dim = da.dims

    # the axes along which to take ffts
    axis_num = [da.get_axis_num(d) for d in dim]

    N = [da.shape[n] for n in axis_num]

    # verify even spacing of input coordinates
    delta_x = []
    for d in dim:
        coord = da[d]
        diff = np.diff(coord)
        if pd.core.common.is_timedelta64_dtype(diff):
            # convert to seconds so we get hertz
            diff = diff.astype('timedelta64[s]').astype('f8')
        delta = diff[0]
        if not np.allclose(diff, diff[0]):
            raise ValueError("Can't take Fourier transform because"
                             "coodinate %s is not evenly spaced" % d)
        delta_x.append(delta)
    # calculate frequencies from coordinates
    k = [ np.fft.fftfreq(Nx, dx) for (Nx, dx) in zip(N, delta_x)]

    if remove_mean:
        da = da - da.mean(dim=dim)

    # the hard work
    #f = np.fft.fftn(da.values, axes=axis_num)
    # need special path for dask
    # is this the best way to check for dask?
    data = da.data
    if hasattr(data, 'dask'):
        assert len(axis_num)==1
        f = dsar.fft.fft(data, axis=axis_num[0])
    else:
        f = np.fft.fftn(data, axes=axis_num)

    if shift:
        f = np.fft.fftshift(f, axes=axis_num)
        k = [np.fft.fftshift(l) for l in k]

    # set up new coordinates for dataarray
    prefix = 'freq_'
    k_names = [prefix + d for d in dim]
    k_coords = {key: val for (key,val) in zip(k_names, k)}

    newdims = list(da.dims)
    for anum, d in zip(axis_num, dim):
        newdims[anum] = prefix + d

    newcoords = {}
    for d in newdims:
        if d in da.coords:
            newcoords[d] = da.coords[d]
        else:
            newcoords[d] = k_coords[d]

    dk = [l[1] - l[0] for l in k]
    for this_dk, d in zip(dk, dim):
        newcoords[prefix + d + '_spacing'] = this_dk

    return xr.DataArray(f, dims=newdims, coords=newcoords)

In [109]:
def power_spectrum(da, dim=None, density=True):
    """
    Perform discrete Fourier transform of xarray data-array `da` along the
    specified dimensions.

    Parameters
    ----------
    da : `xarray.DataArray`
        The data to be transformed
    dim : list (optional)
        The dimensions along which to take the transformation. If `None`, all
        dimensions will be transformed.
    density : list (optional)
        If true, it will normalize the spectrum to PSD
        
    """
    if dim is None:
        dim = da.dims

    # the axes along which to take ffts
    axis_num = [da.get_axis_num(d) for d in dim]

    N = [da.shape[n] for n in axis_num]
    
    daft = dft(da)
    ps = np.real(daft*np.conj(daft))
    
    coord = list(daft.coords)
    if density:
        ps /= (np.asarray(N).prod())**2
        for i in range(int(len(dim))):
            ps /= daft[coord[-i-1]].values


    if density:
        delta_x = []
        for d in dim:
            coord = da[d]
            diff = np.diff(coord)
            if pd.core.common.is_timedelta64_dtype(diff):
                # convert to seconds so we get hertz
                diff = diff.astype('timedelta64[s]').astype('f8')
            delta = diff[0]
            delta_x.append(delta)
        print(delta_x)
        np.testing.assert_almost_equal(ps.sum() 
                                       / (np.asarray(delta_x).prod() 
                                          * (da.values**2).sum()
                                         ), 1., decimal=5
                                      )

    return xr.DataArray(ps, coords=daft.coords, dims=daft.dims)

In [110]:
f = dft(theta, remove_mean=True)
f

/Users/uchidatakaya/anaconda/envs/xrft/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: pandas.core.common.is_timedelta64_dtype is deprecated. import from the public API: pandas.api.types.is_timedelta64_dtype instead
  """Entry point for launching an IPython kernel.


<xarray.DataArray (freq_x: 512, freq_y: 512)>
array([[ 2.523133+0.j      , -2.532964+0.014913j,  0.357434+2.517676j, ...,
        -0.169390+2.547248j,  0.357434-2.517676j, -2.532964-0.014913j],
       [-1.204924+2.228068j,  0.495682-2.494183j,  1.662945-1.937055j, ...,
        -1.356439+2.174617j,  2.393193+0.888929j, -2.443391-0.704621j],
       [ 0.577934-2.476377j,  2.491842+0.555239j, -1.732352+1.888927j, ...,
         2.571527-0.090924j, -2.075890-1.503253j, -1.253123+2.224241j],
       ..., 
       [ 0.155121-2.548157j,  2.434426+0.801533j,  2.178533-1.369311j, ...,
         2.264571+1.243093j, -0.774544+2.453793j, -1.641792+1.968097j],
       [ 0.577934+2.476377j, -1.253123-2.224241j, -2.075890+1.503253j, ...,
        -1.743857+1.892084j, -1.732352-1.888927j,  2.491842-0.555239j],
       [-1.204924-2.228068j, -2.443391+0.704621j,  2.393193-0.888929j, ...,
         1.338718-2.185571j,  1.662945+1.937055j,  0.495682+2.494183j]])
Coordinates:
  * freq_x          (freq_x) float64 -0

In [111]:
psd = power_spectrum(theta)

/Users/uchidatakaya/anaconda/envs/xrft/lib/python3.6/site-packages/ipykernel_launcher.py:25: DeprecationWarning: pandas.core.common.is_timedelta64_dtype is deprecated. import from the public API: pandas.api.types.is_timedelta64_dtype instead


[1, 1]


/Users/uchidatakaya/anaconda/envs/xrft/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: pandas.core.common.is_timedelta64_dtype is deprecated. import from the public API: pandas.api.types.is_timedelta64_dtype instead
  """Entry point for launching an IPython kernel.


In [112]:
psd

<xarray.DataArray (freq_x: 512, freq_y: 512)>
array([[  2.428512e-05,   2.447559e-05,   2.466755e-05, ...,   2.486101e-05,
          2.466755e-05,   2.447559e-05],
       [  2.447559e-05,   2.466831e-05,   2.486254e-05, ...,   2.505831e-05,
          2.486254e-05,   2.466831e-05],
       [  2.466755e-05,   2.486254e-05,   2.505908e-05, ...,   2.525718e-05,
          2.505908e-05,   2.486254e-05],
       ..., 
       [  2.486101e-05,   2.505831e-05,   2.525718e-05, ...,   2.545763e-05,
          2.525718e-05,   2.505831e-05],
       [  2.466755e-05,   2.486254e-05,   2.505908e-05, ...,   2.525718e-05,
          2.505908e-05,   2.486254e-05],
       [  2.447559e-05,   2.466831e-05,   2.486254e-05, ...,   2.505831e-05,
          2.486254e-05,   2.466831e-05]])
Coordinates:
  * freq_x          (freq_x) float64 -0.5 -0.498 -0.4961 -0.4941 -0.4922 ...
  * freq_y          (freq_y) float64 -0.5 -0.498 -0.4961 -0.4941 -0.4922 ...
    freq_x_spacing  float64 0.001953
    freq_y_spacing  float64 

In [119]:
print(psd.dims[0])
k = psd[psd.dims[0]]
k

freq_x


<xarray.DataArray 'freq_x' (freq_x: 512)>
array([-0.5     , -0.498047, -0.496094, ...,  0.494141,  0.496094,  0.498047])
Coordinates:
  * freq_x          (freq_x) float64 -0.5 -0.498 -0.4961 -0.4941 -0.4922 ...
    freq_x_spacing  float64 0.001953
    freq_y_spacing  float64 0.001953